In [48]:
import pandas as pd

train_data = pd.read_csv(r'C:\Users\gabri\Documents\PROJETOS\PY\PJ_Code\DE\Data\Modelo 2\dados_completos - Copy.csv')
test_data = pd.read_csv(r'C:\Users\gabri\Documents\PROJETOS\PY\PJ_Code\DE\Data\test.csv')

In [49]:
# calcula tempo de aprovacao
test_data['order_purchase_timestamp'] = pd.to_datetime(test_data['order_purchase_timestamp'])
test_data['order_approved_at'] = pd.to_datetime(test_data['order_approved_at'])
test_data['TimeToApprove'] = (test_data['order_approved_at'] - test_data['order_purchase_timestamp']).dt.total_seconds() / (60 * 60 * 24)

In [50]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 2.3 * IQR
    upper_bound = Q3 + 2.3 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

train_data = remove_outliers(train_data, 'TimeToApprove')
train_data = train_data.drop_duplicates('order_id', keep='first')
train_data = train_data.dropna(subset=['delivery_time', 'TimeToApprove', 'ApprovedToCarrier','seller_state','distance','product_category_name'])

In [51]:
Q1 = train_data.groupby('seller_state')['delivery_time'].quantile(0.25)
Q3 = train_data.groupby('seller_state')['delivery_time'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

mask = train_data.apply(lambda row: row['delivery_time'] >= lower_bound.get(row['seller_state'], float('-inf')) and row['delivery_time'] <= upper_bound.get(row['seller_state'], float('inf')), axis=1)
train_data = train_data[mask]

In [52]:
import numpy as np
# cria colunas (vazias) que faltam no test
columns_to_add = ['ApprovedToCarrier', 'distance', 'product_category_name', 'seller_state']
for column in columns_to_add:
      test_data[column] = np.nan

In [53]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Initialize the model
numerical_model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(n_estimators=80, random_state=42))
])

# Fit the model on the training data
numerical_model.fit(train_data[['TimeToApprove']], train_data[['ApprovedToCarrier', 'distance']])

# Predict the numerical features for the test dataset
predicted_numerical_features = numerical_model.predict(test_data[['TimeToApprove']])

In [54]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
categorical_model = RandomForestClassifier(n_estimators=80, random_state=42)

# Fit the model on the training data
categorical_model.fit(train_data[['TimeToApprove']], train_data[['product_category_name', 'seller_state']])

# Predict the categorical features for the test dataset
predicted_categorical_features = categorical_model.predict(test_data[['TimeToApprove']])

In [55]:
# Assign predicted numerical features
test_data['ApprovedToCarrier'] = predicted_numerical_features[:, 0]
test_data['distance'] = predicted_numerical_features[:, 1]

# Assign predicted categorical features
test_data['product_category_name'] = predicted_categorical_features[:, 0]
test_data['seller_state'] = predicted_categorical_features[:, 1]

In [56]:
test_data.to_csv(r'C:\Users\gabri\Documents\PROJETOS\PY\PJ_Code\DE\Data\Modelo 2\teste2_menosFULL.csv', index=False)